# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrea.Xue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrea.Xue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andrea.Xue\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrea.Xue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_message',con=engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.shape[0]

26216

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    clean_tokens = []
    for sent in text:
        # remove non-alphebetic contents, including punctuations, numbers
        sent = re.sub(r"[^a-zA-Z]", " ", sent)
        tokens = word_tokenize(sent)
        lemmatizer = WordNetLemmatizer()
        # lemmatize and remove stop words
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            if clean_tok not in stopwords.words("english"):
                clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:
len(tokenize(X))

371440

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, n_jobs =-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
x_train,x_test, y_train,y_test = train_test_split(X, Y, test_size=0.3)

In [32]:
pipeline.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [0]:
y_pred = pipeline.predict(x_test)

In [34]:
from sklearn.metrics import classification_report, accuracy_score
for i in range(y_test.shape[1]):
    print("Category Name: {}".format(df.iloc[:,4:].columns[i]))
    print(classification_report(y_test[:,i], y_pred[:,i]))

Category Name: related
              precision    recall  f1-score   support

           0       0.58      0.16      0.25      1869
           1       0.78      0.96      0.86      5939
           2       0.50      0.04      0.07        57

    accuracy                           0.77      7865
   macro avg       0.62      0.39      0.39      7865
weighted avg       0.73      0.77      0.71      7865

Category Name: request
              precision    recall  f1-score   support

           0       0.85      0.99      0.91      6542
           1       0.66      0.14      0.23      1323

    accuracy                           0.84      7865
   macro avg       0.76      0.56      0.57      7865
weighted avg       0.82      0.84      0.80      7865

Category Name: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7820
           1       0.00      0.00      0.00        45

    accuracy                           0.99      7865
   macr

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.91      1.00      0.96      7181
           1       0.74      0.02      0.04       684

    accuracy                           0.91      7865
   macro avg       0.83      0.51      0.50      7865
weighted avg       0.90      0.91      0.88      7865

Category Name: clothing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7727
           1       0.50      0.01      0.01       138

    accuracy                           0.98      7865
   macro avg       0.74      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

Category Name: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7686
           1       1.00      0.01      0.01       179

    accuracy                           0.98      7865
   macro avg       0.99      0.50      0.50      7865
weighted avg       0.98      

In [9]:
def build_report(y_pred, avg_param):
    
    # build classification report on every column
    performances = []
    for i in range(len(df.iloc[:,4:].columns)):
        performances.append([f1_score(y_test[:, i], y_pred[:, i], average= avg_param),
                             precision_score(y_test[:, i], y_pred[:, i], average= avg_param),
                             recall_score(y_test[:, i], y_pred[:, i], average= avg_param)])
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = df.iloc[:,4:].columns)   
    return performances

In [35]:
micro_perform = build_report(y_pred,avg_param = 'micro')
micro_perform

,f1 score,precision,recall
related,0.766561,0.766561,0.766561
request,0.843357,0.843357,0.843357
offer,0.994278,0.994278,0.994278
aid_related,0.636999,0.636999,0.636999
medical_help,0.919263,0.919263,0.919263
medical_products,0.948252,0.948252,0.948252
search_and_rescue,0.973172,0.973172,0.973172
security,0.982327,0.982327,0.982327
military,0.966688,0.966688,0.966688
child_alone,1.000000,1.000000,1.000000


In [36]:
weighted_perform = build_report(y_pred,avg_param = 'weighted')
weighted_perform

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,f1 score,precision,recall
related,0.711289,0.730016,0.766561
request,0.798405,0.818313,0.843357
offer,0.991426,0.988590,0.994278
aid_related,0.620828,0.627944,0.636999
medical_help,0.882536,0.881392,0.919263
medical_products,0.923807,0.917032,0.948252
search_and_rescue,0.959941,0.947064,0.973172
security,0.973695,0.965213,0.982327
military,0.950564,0.934969,0.966688
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
def tokenize(text):
    clean_tokens = []
    for sent in text:
        # remove non-alphebetic contents, including punctuations, numbers
        sent = re.sub(r"[^a-zA-Z]", " ", sent)
        tokens = word_tokenize(sent)
        lemmatizer = WordNetLemmatizer()
        # lemmatize and remove stop words
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            if clean_tok not in stopwords.words("english"):
                clean_tokens.append(clean_tok)

    return clean_tokens

pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 80)))
])
parameters = [{'vect__max_features': [50000, 80000],
              #'vect__use_idf': [True, False],
              'clf__estimator__max_depth': [10,15]
              }]

grid_cv1 = GridSearchCV(pipeline, parameters, cv=3, verbose=3, n_jobs = 10)

In [16]:
grid_cv1.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=10)]: Done   3 out of  12 | elapsed: 85.5min remaining: 256.5min
[Parallel(n_jobs=10)]: Done   8 out of  12 | elapsed: 96.7min remaining: 48.3min
[Parallel(n_jobs=10)]: Done  12 out of  12 | elapsed: 111.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
grid_cv1.best_params_

{'clf__estimator__max_depth': 15, 'vect__max_features': 80000}

In [0]:
best_estimator = grid_cv1.best_estimator_
y_pred_cv = best_estimator.predict(x_test)

In [22]:
weighted_perform_cv = build_report(y_pred_cv, avg_param = 'micro')
weighted_perform_cv

,f1 score,precision,recall
related,0.775969,0.775969,0.775969
request,0.847807,0.847807,0.847807
offer,0.995168,0.995168,0.995168
aid_related,0.634329,0.634329,0.634329
medical_help,0.922187,0.922187,0.922187
medical_products,0.945963,0.945963,0.945963
search_and_rescue,0.972155,0.972155,0.972155
security,0.983090,0.983090,0.983090
military,0.967196,0.967196,0.967196
child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [0]:
from xgboost import XGBClassifier

In [0]:
pipeline_new = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize, max_features = 80000)),
    ('clf', MultiOutputClassifier(XGBClassifier(n_estimators = 100, max_depth=12, colsample_bytree=0.5, verbosity=3, n_jobs=10)))
])
                           
pipeline_new.fit(x_train, y_train)

In [29]:
y_pred_xgb = pipeline_new.predict(x_test)
build_report(y_pred_xgb, 'micro')

,f1 score,precision,recall
related,0.776987,0.776987,0.776987
request,0.848188,0.848188,0.848188
offer,0.995168,0.995168,0.995168
aid_related,0.631532,0.631532,0.631532
medical_help,0.921933,0.921933,0.921933
medical_products,0.945709,0.945709,0.945709
search_and_rescue,0.972028,0.972028,0.972028
security,0.983090,0.983090,0.983090
military,0.967196,0.967196,0.967196
child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [0]:
import pickle
pickle.dump(pipeline_new, open('xgb_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.